In [5]:

from sympy import symbols, Eq, And, Xor
from sympy.logic.inference import satisfiable
import re

# Define symbols for houses
houses = 2
house_colours = symbols(f'house_colour:{houses}')
house_owners = symbols(f'house_owner:{houses}')
house_drinks = symbols(f'house_drink:{houses}')

# Specific entities
colours = (blue, green, red, yellow, purple) = symbols('blue green red yellow purple')
nationalities = (canadian, american, japanese, british, german) = symbols('canadian american japanese british german')
drinks = (coffee, tea, milk, juice, water) = symbols('coffee tea milk juice water')

# now for constraints
constraints = []

# 1:1 houses:nationalites
for i in range(houses):
    constraints.append(Xor(*[Eq(house_owners[i], nationality) for nationality in nationalities[:houses]]))

# 1:1 houses:colours
for i in range(houses):
    constraints.append(Xor(*[Eq(house_colours[i], colour) for colour in colours[:houses]]))

# 1:1 houses:drinks
for i in range(houses):
    constraints.append(Xor(*[Eq(house_drinks[i], drink) for drink in drinks[:houses]]))

# Ensure each owner is assigned to exactly one house
for nationality in nationalities[:houses]:
    constraints.append(Xor(*[Eq(house_owners[i], nationality) for i in range(houses)]))

# Ensure each colour is assigned to exactly one house
for colour in colours[:houses]:
    constraints.append(Xor(*[Eq(house_colours[i], colour) for i in range(houses)]))

# Ensure each drink is assigned to exactly one house
for drink in drinks[:houses]:
    constraints.append(Xor(*[Eq(house_drinks[i], drink) for i in range(houses)]))
    
# the blue house has tea
constraints.append(Xor(*[(Eq(house_colours[i], blue) & Eq(house_drinks[i], tea)) for i in range(houses)]))

# only the canadian owns the house which has coffee
constraints.append(Xor(*[(Eq(house_owners[i], canadian) & Eq(house_drinks[i], coffee)) for i in range(houses)]))

# Add constraints to prevent the same house from having two different colours, owners, or drinks
constraints.append(~(Eq(house_colours[0], blue) & Eq(house_colours[0], green)))
constraints.append(~(Eq(house_colours[1], blue) & Eq(house_colours[1], green)))
constraints.append(~(Eq(house_owners[0], american) & Eq(house_owners[0], canadian)))
constraints.append(~(Eq(house_owners[1], american) & Eq(house_owners[1], canadian)))
constraints.append(~(Eq(house_drinks[0], coffee) & Eq(house_drinks[0], tea)))
constraints.append(~(Eq(house_drinks[1], coffee) & Eq(house_drinks[1], tea)))

# Combine constraints into one logical expression using And
combined_constraints = And(*constraints)

# Check satisfiability
result = satisfiable(combined_constraints)

houses = {}

def print_result(result):
    # Process each entry in the SymPy result
    for attr, value in [(k.lhs, k.rhs) for k,v in result.items() if v]:
        # Extract the attribute type and house number from the symbol name
        attr_name = str(attr)
        if 'owner' in attr_name:
            attr_type = 'owner'
        elif 'drink' in attr_name:
            attr_type = 'drink'
        elif 'colour' in attr_name:
            attr_type = 'colour'
        house_num = int(re.search(r'(\d+)$', attr_name).group(1))
        
        # Initialize the house entry if not already present
        if house_num not in houses:
            houses[house_num] = {"owner": None, "drink": None, "colour": None}
        
        # Assign the value to the corresponding attribute
        houses[house_num][attr_type] = value

    # Format and print the results
    for house_num in sorted(houses):
        owner = houses[house_num]["owner"]
        colour = houses[house_num]["colour"]
        drink = houses[house_num]["drink"]
        print(f"{owner}, {colour}, {drink}")

if result: 
    print_result(result)
else:
    print("No solution found")

canadian, green, coffee
american, blue, tea
